# Accurate Integer Addition in Transformers - Analyse the Model

This CoLab analyses a Transformer model that performs integer addition e.g. 33357+82243=115600. Each digit is a separate token. For 5 digit addition, the model is given 12 "question" (input) tokens, and must then predict the corresponding 6 "answer" (output) tokens.

The model weightings created by the sister CoLab [Accurate_Addition_Train](https://github.com/PhilipQuirke/transformer-maths/blob/main/assets/Accurate_Addition_Train.ipynb) are loaded from Google Drive.

## Tips for using the Colab
 * You can run and alter the code in this CoLab notebook yourself in Google CoLab ( https://colab.research.google.com/ ).
 * To run the notebook, in Google CoLab, **you will need to** go to Runtime > Change Runtime Type and select GPU as the hardware accelerator.
 * Some graphs are interactive!
 * Use the table of contents pane in the sidebar to navigate.
 * Collapse irrelevant sections with the dropdown arrows.
 * Search the page using the search in the sidebar, not CTRL+F.

# Part 1: Configuration


In [79]:
# Tokens used in vocab. (Token indexes 0 to 9 represent digits 0 to 9)
PLUS_INDEX = 10
MINUS_INDEX = 11
EQUALS_INDEX = 12

class Config():
  #@markdown Model
  n_layers: int = 2 #@param
  n_heads: int = 3 #@param

  d_vocab: int = EQUALS_INDEX+1
  d_model: int = ( 512 // n_heads ) * n_heads # About 512, and divisible by n_heads
  d_mlp: int = 4 * d_model
  d_head: int = d_model // n_heads  # About 170 when n_heads == 3
  seed: int = 129000 #@param

  #@markdown Data
  n_digits: int = 5 #@param
  n_ctx: int = 3 * n_digits + 3
  act_fn: str = 'relu'
  batch_size: int = 64 #@param

  #@markdown Optimizer
  n_training_steps: int = 30000 #@param
  lr: float = 0.00008 #@param
  weight_decay: int = 0.1 #@param

  # Save graphs to CoLab temp files as PDF and HTML. Can manually export files for re-use in papers.
  save_graph_to_file: bool = True

  # The format to output prettytable in. Options are text|html|json|csv|latex
  # Use Text for this CoLab, latex for Overleaf output, and html for GitHub blog output
  table_out_format: str = "text"


cfg = Config()

# Part 2: Import libraries
Imports standard libraries. Don't bother reading.
You will need to give permission for this CoLab to access your Google Drive to load the model weights (created by the "Accurate Addition - Train" CoLab.

In [80]:
from google.colab import drive
from pathlib import Path

In [81]:
GLOBAL=True
if GLOBAL:
    drive.mount('/content/drive', force_remount=False)
    rootdir=Path('/content/drive/MyDrive/AI/CoLabOutput/')
else:
    rootdir=Path('./')

base_fname = '_digits{}_layer{}_heads{}_dmodel{}_dhead{}_ctx{}_seed{}_train{}.pt'.format(cfg.n_digits, cfg.n_layers, cfg.n_heads, cfg.d_model, cfg.d_head, cfg.n_ctx, cfg.seed, cfg.n_training_steps)

add_fname = 'add' + base_fname
model_save_location = rootdir/f'{add_fname}'

print(f'rootdir is {rootdir}')
print('model will save to {}'.format(str(model_save_location)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
rootdir is /content/drive/MyDrive/AI/CoLabOutput
model will save to /content/drive/MyDrive/AI/CoLabOutput/add_digits5_layer2_heads3_dmodel510_dhead170_ctx18_seed129000_train30000.pt


In [ ]:
DEVELOPMENT_MODE = True
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")

    %pip install --upgrade numpy
    %pip install matplotlib
    %pip install prettytable
    %pip install seaborn

    %pip install kaleido
    %pip install transformer_lens
    %pip install torchtyping
    %pip install transformers

except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook


In [ ]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import kaleido
import plotly.io as pio

if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
pio.templates['plotly'].layout.xaxis.title.font.size = 20
pio.templates['plotly'].layout.yaxis.title.font.size = 20
pio.templates['plotly'].layout.title.font.size = 30

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import tqdm.auto as tqdm
import random
from prettytable import PrettyTable
import matplotlib.pyplot as plt


# Use seaborn library to display heatmaps
use_sns= True
try:
  import seaborn as sns
except Exception as e:
  print("sns import exception", e)
  use_sns = False

# Use Principal Component Analysis (PCA) library
use_pca = True
try:
  from sklearn.decomposition import PCA
except Exception as e:
  print("pca import exception", e)
  use_pca = False

In [ ]:
import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache


# Part 3: Create model
This section defines the token embedding / unembedding and creates the model.

In [ ]:
# Embedding / Unembedding

def tokens_to_string(tokens):
    tokens = utils.to_numpy(tokens)
    x = "".join([str(i) for i in tokens[:cfg.n_digits]])
    y = "".join([str(i) for i in tokens[cfg.n_digits+1:cfg.n_digits*2+1]])
    z = "".join([str(i) for i in tokens[cfg.n_ctx-cfg.n_digits-1:]])
    equals = "="
    operator = "+"
    return f"{x}{operator}{y}{equals}{z}"

def string_to_tokens(string, batch: bool=False):
    lookup = {str(i):i for i in range(10)}
    lookup['+']=PLUS_INDEX
    lookup['-']=MINUS_INDEX
    lookup['=']=EQUALS_INDEX

    tokens = [lookup[i] for i in string if i not in '\n ']
    if batch:
        return torch.tensor(tokens)[None, :]
    else:
        return torch.tensor(tokens)

In [ ]:
# Transformer creation

# Structure is documented at https://neelnanda-io.github.io/TransformerLens/transformer_lens.html#transformer_lens.HookedTransformerConfig.HookedTransformerConfig
ht_cfg = HookedTransformerConfig(
    n_layers = cfg.n_layers,
    n_heads = cfg.n_heads,
    d_model = cfg.d_model,
    d_head = cfg.d_head,
    d_mlp = cfg.d_mlp,
    act_fn = cfg.act_fn,
    normalization_type = 'LN',
    d_vocab = cfg.d_vocab,
    d_vocab_out = cfg.d_vocab,
    n_ctx = cfg.n_ctx,
    init_weights = True,
    device = "cuda",
    seed = cfg.seed,
)

model = HookedTransformer(ht_cfg)

optimizer = optim.AdamW(model.parameters(),
                        lr = cfg.lr,
                        weight_decay = cfg.weight_decay,
                        betas = (0.9, 0.98))

# Gives 2e-9 loss (but changes the cells used by the model)
# max_iter = cfg.n_training_steps
# warmup_iter = max_iter // 5
# scheduler1 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.01, total_iters=int(warmup_iter))
# scheduler2 = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=int(np.ceil((max_iter-warmup_iter))))
# scheduler  = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2], milestones=[int(warmup_iter)])

# Gives 3e-8 loss
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda step: min(step/10, 1))

# Part 4: Data Generator. Addition sub-task categorisation
This section defines the loss function and the training/tesing data generator.

It also defines functions to categorise the training data by the addition sub-task defined in the paper. The addition sub tasks are abbreviated as:
- BA is Base Add. Calculates the sum of two digits Dn and Dn' modulo 10, ignoring any carry over from previous columns.
- MC1 is Make Carry 1. Evaluates to true if adding digits Dn and Dn' results in a carry over of 1 to the next column.
- MS9 is Make Sum 9. Evaluates to true if adding digits Dn and Dn' gives exactly 9.
- UC1 is Use Carry 1. Takes the previous column's carry output and adds it to the sum of the current digit pair.
- US9 is Use Sum 9. Propagates (aka cascades) a carry over of 1 to the next column if the current column sums to 9 and the previous column generated a carry over. US9 is the most complex task as it spans three digits. For some rare questions (e.g. 00555 + 00445 = 01000) US9 applies to up to four sequential digits, causing a chain effect, with the MC1 cascading through multiple digits.

In [ ]:
# Loss functions

# Calculate the per-token probability by comparing a batch of prediction "logits" to answer "tokens"
def logits_to_tokens_loss(logits, tokens):

  # The last "n_digit+1" tokens are the addition answer probabilities
  ans_logits = logits[:, -(cfg.n_digits+2):-1]

  # Convert raw score (logits) vector into a probability distribution.
  # Emphasize the largest scores and suppress the smaller ones, to make them more distinguishable.
  ans_probs = F.log_softmax(ans_logits.to(torch.float64), dim=-1)

  max_indices = torch.argmax(ans_probs, dim=-1)

  # The last "n_digit+1" tokens are the model’s answer.
  ans_tokens = tokens[:, -(cfg.n_digits+1):]

  # Extract values from the ans_probs tensor, based on indices from the ans_tokens tensor
  ans_loss = torch.gather(ans_probs, -1, ans_tokens[:, :, None])[..., 0]

  return ans_loss, max_indices

# Calculate loss as negative of average per-token mean probability
def loss_fn(ans_loss):
  return -ans_loss.mean(0)

In [ ]:
# Define "iterator" data generator function. Invoked using next().
# "Addition" batch entries are formated XXXXX+YYYYY=ZZZZZZ e.g. 55003+80002=135005
# "Subtraction" batch entries are formated XXXXX-YYYYY=ZZZZZZ e.g. 55003-80002=-24999, 80002-55003=024999
# Note that answer has one more digit than the question
# Returns characteristics of each batch entry to aid later analysis
def data_generator():
    torch.manual_seed(cfg.seed)
    while True:
        #generate a batch of questions (answers calculated below)
        batch = torch.zeros((cfg.batch_size, cfg.n_ctx)).to(torch.int64)
        x = torch.randint(0, 10, (cfg.batch_size, cfg.n_digits))
        y = torch.randint(0, 10, (cfg.batch_size, cfg.n_digits))


        # The UseSum9 task is compound and rare and so hard to learn.
        # For some batches, we increase the MakeSum9 case frequency
        # UseSum9 also relies on MakeCarry1 (50%) from previous column.
        # So UseSum9 frequency is increased by 60% * 40% * 50% = 12%
        if random.randint(1, 5) < 3: # 60%
          # Flatten x and y to 1D tensors
          x_flat = x.view(-1)
          y_flat = y.view(-1)

          num_elements_to_modify = int(0.40 * x.numel()) # 40%
          indices_to_modify = torch.randperm(x_flat.numel())[:num_elements_to_modify]
          if random.randint(1, 2) == 1:
            x_flat[indices_to_modify] = 9 - y_flat[indices_to_modify]
          else:
            y_flat[indices_to_modify] = 9 - x_flat[indices_to_modify]

          # Reshape x and y back to its original shape
          x = x_flat.view(x.shape)
          y = y_flat.view(x.shape)


        batch[:, :cfg.n_digits] = x
        batch[:, cfg.n_digits] = PLUS_INDEX
        batch[:, 1+cfg.n_digits:1+cfg.n_digits*2] = y
        batch[:, 1+cfg.n_digits*2] = EQUALS_INDEX

        # These attributes are used for testing addition
        base_adds = torch.zeros((cfg.batch_size,cfg.n_digits)).to(torch.int64)
        make_carry1s = torch.zeros((cfg.batch_size,cfg.n_digits)).to(torch.int64)
        sum9s = torch.zeros((cfg.batch_size,cfg.n_digits)).to(torch.int64)
        use_carry1s = torch.zeros((cfg.batch_size,cfg.n_digits)).to(torch.int64)
        use_sum9s = torch.zeros((cfg.batch_size,cfg.n_digits)).to(torch.int64)

        # generate the addition question answers & other info for testing
        for i in range(cfg.n_digits):
            # the column in the test attributes being updated
            test_col = cfg.n_digits-1-i

            base_add = batch[:, cfg.n_digits-1-i] + batch[:, 2*cfg.n_digits-i]
            base_adds[:, test_col] = base_add % 10

            sum9 = (base_add == 9)
            sum9s[:, test_col] = sum9

            if i>0:
              use_carry1s[:, test_col] = make_carry1s[:, test_col+1]
            use_carry = use_carry1s[:, test_col]

            use_sum9s[:, test_col] = sum9 & use_carry;

            digit_sum = base_add + use_carry1s[:, test_col]

            make_carry = (digit_sum >= 10)
            make_carry1s[:, test_col] = make_carry

            batch[:, -1-i] = (digit_sum % 10)

        # Final (possible) carry to highest digit of the sum
        batch[:, -1-cfg.n_digits] = make_carry1s[:, 0]

        yield batch.cuda(), base_adds.cuda(), make_carry1s.cuda(), sum9s.cuda(), use_carry1s.cuda(), use_sum9s.cuda()

In [ ]:
ds = data_generator()

tokens, base_adds, make_carry1s, sum9s, use_carry1s, use_sum9s = next(ds)

print(tokens[0])

# Part 5 Load Model from Google Drive

In [ ]:
def print_config():
  print("n_digits=", cfg.n_digits, "n_heads=", cfg.n_heads, "n_layers=", cfg.n_layers, "n_ctx=", cfg.n_ctx, "seed=", cfg.seed, "n_training_steps=", cfg.n_training_steps)

In [ ]:
print("Loading model from file", model_save_location)
model.load_state_dict(torch.load(model_save_location))
model.eval()

# Part 8: Sample Questions Set Up

Create sets of sample questions (by task) to ask the model to predict

In [ ]:
# Insert a number into the question
def insert_question_number(the_question, index, first_digit_index, the_digits, n):

  last_digit_index = first_digit_index + the_digits - 1

  for j in range(the_digits):
    the_question[index, last_digit_index-j] = n % 10
    n = n // 10


# Create a single question
def make_a_question(the_question, index, q1, q2):
  a = q1 + q2

  insert_question_number(the_question, index, 0, cfg.n_digits, q1)

  the_question[index, cfg.n_digits] = PLUS_INDEX

  insert_question_number( the_question, index, cfg.n_digits+1, cfg.n_digits, q2)

  the_question[index, 2*cfg.n_digits+1] = EQUALS_INDEX
  offset = 2

  insert_question_number(the_question, index, 2*cfg.n_digits + offset, cfg.n_digits+1, q1+q2)


# Create a batch of questions from a 2D matrix of ints
def make_questions(q_matrix):
  length = len(q_matrix)

  questions = torch.zeros((length, cfg.n_ctx)).to(torch.int64)

  limit = 10 ** cfg.n_digits
  for i in range(length):
    if (q_matrix[i][0] < limit) and (q_matrix[i][1] < limit) :
      make_a_question(questions, i, q_matrix[i][0], q_matrix[i][1])

  return questions


def prediction_to_string(max_indices):
  answer = "".join([str(i) for i in utils.to_numpy(max_indices)[0]])
  return answer;

In [ ]:
# Analyse the question and return the use case as BA, MC, SimpleUS9 or CascadeUS9
def get_question_case(q):
  qa = utils.to_numpy(q)
  qn = qa[:2*cfg.n_digits+2]

  # Locate the MC and MS digits (if any)
  mc = torch.zeros( cfg.n_digits).to(torch.int64)
  ms = torch.zeros( cfg.n_digits).to(torch.int64)
  for dn in range(cfg.n_digits):
    if qn[dn] + qn[dn + cfg.n_digits + 1] == 9:
      ms[cfg.n_digits-1-dn] = 1
    if qn[dn] + qn[dn + cfg.n_digits +1] > 9:
      mc[cfg.n_digits-1-dn] = 1

  # Calculate the use case of a question
  if torch.sum(mc) == 0:
    return "BA"

  if torch.sum(ms) == 0:
    return "MC1"

  for dn in range(cfg.n_digits):
    if dn < cfg.n_digits-2 and mc[dn] == 1 and ms[dn+1] == 1 and ms[dn+2] == 1:
      return "CascadeUS9"

  for dn in range(cfg.n_digits):
    if dn < cfg.n_digits-1 and mc[dn] == 1 and ms[dn+1] == 1:
      return "SimpleUS9"

  return "MC1"

In [ ]:
# Manually create some questions that strongly test one use case


def make_ba_questions():
    return make_questions(
      [[12345, 33333],
      [33333, 12345],
      [45762, 33113],
      [888, 11111],
      [2362, 23123],
      [15, 81],
      [1000, 4440],
      [4440, 1000],
      [24033, 25133],
      [23533, 21133],
      [32500, 1],
      [31500, 1111],
      [5500, 12323],
      [4500, 2209],
      [ 33345, 66643], # =099988
      [ 66643, 33345], # =099988
      [10990, 44000],
      [60000, 30000],
      [10000, 20000]])


def make_uc1_questions():
    return make_questions(
      [[ 15, 45],
      [ 25, 55],
      [ 35, 59],
      [ 40035, 40049],
      [ 5025, 5059],
      [ 15, 65],
      [ 44000, 46000],
      [ 70000, 40000],
      [ 15000, 25000],
      [ 35000, 35000],
      [ 45000, 85000],
      [ 67000, 85000],
      [ 99000, 76000],
      [ 1500, 4500],
      [ 2500, 5500],
      [ 3500, 5900],
      [ 15020, 45091],
      [ 25002, 55019],
      [ 35002, 59019]])


def make_simple_us9_questions():
    return make_questions(
      [[ 55, 45],
      [ 45, 55],
      [ 45, 59],
      [ 35, 69],
      [ 25, 79],
      [ 15, 85],
      [ 15, 88],
      [ 15508, 14500],
      [ 14508, 15500],
      [ 24533, 25933],
      [ 23533, 26933],
      [ 32500, 7900],
      [ 31500, 8500],
      [ 550, 450],
      [ 450, 550],
      [ 10880, 41127],
      [ 41127, 10880],
      [ 12386, 82623]])


def make_cascade_us9_questions(clean = True):
    return make_questions(
      # These are two level UseSum9 cascades
      [[ 555, 445],
      [ 3340, 6660],
      [ 8880, 1120],
      [ 1120, 8880],
      [ 123, 877],
      [ 877, 123],
      [ 321, 679],
      [ 679, 321],
      [ 1283, 88786],
      # These are three level UseSum9 cascades
      [ 5555, 4445],
      [ 55550, 44450],
      [ 334, 666],
      [ 3340, 6660],
      [ 33400, 66600],
      [ 888, 112],
      [ 8880, 1120],
      [ 88800, 11200],
      [ 1234, 8766],
      [ 4321, 5679],
      # These are four level UseSum9 cascades
      [ 44445, 55555],
      [ 33334, 66666],
      [ 88888, 11112],
      [ 12345, 87655],
      [ 54321, 45679],
      [ 45545, 54455],
      [ 36634, 63366],
      [ 81818, 18182],
      [ 87345, 12655],
      [ 55379, 44621]])


# These questions focus mainly on 1 digit at a time
# (We're assuming that the 0 + 0 digit additions are trivial bigrams)
def make_answerdigit_questions():
    return make_questions(
      [[ 1, 0],
      [ 4, 3],
      [ 5, 5],
      [ 8, 1],
      [ 40, 30],
      [ 44, 46],
      [ 400, 300],
      [ 440, 460],
      [ 800, 100],
      [ 270, 470],
      [ 600, 300],
      [ 4000, 3000],
      [ 4400, 4600],
      [ 6000, 3000],
      [ 7000, 4000],
      [ 40000, 30000],
      [ 44000, 46000],
      [ 60000, 30000],
      [ 70000, 40000],
      [ 10000, 20000],
      [ 15000, 25000],
      [ 35000, 35000],
      [ 45000, 85000],
      [ 67000, 85000],
      [ 99000, 76000],
      [ 76000, 99000]])


# Returns 128 random and ~100 manually-chosen questions
def make_varied_questions():
  q0, _, _, _, _, _ = next(ds)
  q1 = make_ba_questions()
  q2 = make_uc1_questions()
  q3 = make_simple_us9_questions()
  q4 = make_cascade_us9_questions()
  q5 = make_answerdigit_questions()
  q6, _, _, _, _, _ = next(ds)

  questions = torch.vstack((q0.cuda(), q1.cuda(), q2.cuda(), q3.cuda(), q4.cuda(), q5.cuda(), q6.cuda()))

  return questions

In [ ]:
# Test that the get_question_case code works as expected
def unit_test_get_question_case_core(correct_case, questions):
  num_questions = questions.shape[0]
  print( correct_case, "#Questions=", num_questions)
  for i in range(num_questions):
    question_case = get_question_case(questions[i])
    if question_case != correct_case:
      print( "Case mismatch:", correct_case, question_case, questions[i])

def unit_test_get_question_case():
  unit_test_get_question_case_core( "BA", make_ba_questions())
  unit_test_get_question_case_core( "MC1", make_uc1_questions())
  unit_test_get_question_case_core( "SimpleUS9", make_simple_us9_questions())
  unit_test_get_question_case_core( "CascadeUS9", make_cascade_us9_questions())

unit_test_get_question_case()

In [ ]:
num_questions = 0;
correct_answers = 0;
verbose = True
total_mean_loss = 0.0


# Clear the question summary results
def clear_questions_results(title):
  global num_questions
  global correct_answers
  global verbose
  global total_mean_loss

  num_questions = 0
  correct_answers = 0
  total_mean_loss = 0

  if verbose:
    print(title)


# Ask model to predict answer for each question & collect results
def do_questions(questions):
    global num_questions
    global correct_answers
    global verbose
    global total_mean_loss

    num_questions = questions.shape[0]
    for question_num in range(num_questions):
      q = questions[question_num]

      # Run with no hook
      the_logits = model(q.cuda())

      q_2d = q.unsqueeze(0)
      losses_raw, max_indices = logits_to_tokens_loss(the_logits, q_2d.cuda())
      losses = loss_fn(losses_raw)
      mean_loss = utils.to_numpy(losses.mean())
      total_mean_loss = total_mean_loss + mean_loss

      model_answer_str = prediction_to_string(max_indices)
      model_answer_num = int(model_answer_str)

      i = cfg.n_digits*2 + 2

      a = 0
      # 5 digit addition yields a 6 digit answer. Hence cfg.n_digits+1
      for j in range(cfg.n_digits+1):
        a = a * 10 + q[i+j]

      correct = (model_answer_num == a)
      if correct :
        correct_answers += 1

      if verbose:
        print(tokens_to_string(q), "ModelAnswer:", model_answer_str, "Correct:", correct, "Loss:", mean_loss )


# Print the question summary results
def print_questions_results(prefix, output_table):
  global num_questions
  global correct_answers
  global total_mean_loss

  output_table.add_row([prefix, num_questions, str(correct_answers), 100*correct_answers/num_questions, total_mean_loss/num_questions])

In [ ]:
# Build a test batch of 64 random and ~100 manually-chosen questions
varied_questions = make_varied_questions();


# Run the sample batch, gather the cache
model.reset_hooks()
model.set_use_attn_result(True)
sample_logits, sample_cache = model.run_with_cache(varied_questions.cuda())
print(sample_cache) # Gives names of datasets in the cache
sample_losses_raw, sample_max_indices = logits_to_tokens_loss(sample_logits, varied_questions.cuda())
sample_loss_mean = utils.to_numpy(loss_fn(sample_losses_raw).mean())
print("Sample Mean Loss", sample_loss_mean) # Loss < 0.04 is good


# attn.hook_z is the "attention head output" hook point name (at a specified layer)
# Used in h_set_attn_hook_z and t_*_hook functions
l_attn_hook_z_name = ['blocks.0.attn.hook_z','blocks.1.attn.hook_z']
sample_attn_z = sample_cache[l_attn_hook_z_name[0]]
print("Sample", l_attn_hook_z_name[0], sample_attn_z.shape) # gives [239, 18, 3, 170] = #questions, cfg.n_ctx, n_heads, d_head
mean_attn_z = torch.mean(sample_attn_z, dim=0, keepdim=True)
print("Mean", l_attn_hook_z_name[0], mean_attn_z.shape) # gives [1, 18, 3, 170] = 1, cfg.n_ctx, n_heads, d_head


# hook_resid_pre is the "pre residual memory update" hook point name (at a specified layer)
# Used in o_*_hook functions
l_hook_resid_pre_name = ['blocks.0.hook_resid_pre','blocks.1.hook_resid_pre']


# hook_resid_post is the "post residual memory update" hook point name (at a specified layer)
# Used in c_*_hook and t_*_hook functions
l_hook_resid_post_name = ['blocks.0.hook_resid_post','blocks.1.hook_resid_post']
sample_resid_post = sample_cache[l_hook_resid_post_name[0]]
print("Sample", l_hook_resid_post_name[0], sample_resid_post.shape) # gives [239, 18, 510] = #questions, cfg.n_ctx, d_model
mean_resid_post = torch.mean(sample_resid_post, dim=0, keepdim=True)
print("Mean", l_hook_resid_post_name[0], mean_resid_post.shape) # gives [1, 18, 510] = 1, cfg.n_ctx, d_model


# mlp.hook_post is the "MLP layer" hook point name (at a specified layer)
# Used in m_*_hook functions
l_mlp_hook_post_name = ['blocks.0.mlp.hook_post','blocks.1.mlp.hook_post']
sample_mlp_hook_post = sample_cache[l_mlp_hook_post_name[0]]
print("Sample", l_mlp_hook_post_name[0], sample_mlp_hook_post.shape) # gives [239, 18, 2040] = #questions, cfg.n_ctx, d_model*4
mean_mlp_hook_post = torch.mean(sample_mlp_hook_post, dim=0, keepdim=True)
print("Mean", l_mlp_hook_post_name[0], mean_mlp_hook_post.shape) # gives [1, 18, 2040] = 1, cfg.n_ctx, d_model*4

# Part 9: Prediction Analysis By Use Case
This section sets up BA, UC1 and US9 test cases that will be re-used in later experiments to show the impact of ablating heads or token positions.

In [ ]:
exp0_output = PrettyTable()
exp0_output.field_names = ["Case", "#Questions", "#Correct", "%Correct", "Mean loss"]
verbose = False

clear_questions_results("Simple BaseAdd cases")
do_questions(make_ba_questions())
print_questions_results("BaseAdd", exp0_output)
sum_total_mean_loss = total_mean_loss
sum_num_questions = num_questions

clear_questions_results("These are Use Carry 1 (UC1) examples (not UseSum9 examples)")
do_questions(make_uc1_questions())
print_questions_results("UseCarry1", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

clear_questions_results("These are simple (one level) UseSum9 exampless")
do_questions(make_simple_us9_questions())
print_questions_results("SimpleUS9", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

clear_questions_results("These are UseSum9 two, three and four level cascades")
do_questions(make_cascade_us9_questions())
print_questions_results("CascadeUS9", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

clear_questions_results("These questions focus on different answer digits")
do_questions(make_answerdigit_questions())
print_questions_results("AnswerDigits", exp0_output)
sum_total_mean_loss = sum_total_mean_loss + total_mean_loss
sum_num_questions = sum_num_questions + num_questions

exp0_output.add_row(["OVERALL", sum_num_questions, "", "", sum_total_mean_loss])

print_config()
print()
print(exp0_output.get_formatted_string(out_format=cfg.table_out_format))

# Part 11: Set Up "Count" Framework

Create way to get model to predict sample question answers and analysis/show results. Use prefix "c_"

In [ ]:
# Build up a list of success/failure by case (BA, MC1, US9) found, and the frequency of each case
c_case_counts = {}


def count_question_cases(questions):
  global c_case_counts

  c_case_counts = {}

  for i in range(questions.shape[0]):
    q_case = get_question_case(questions[i])

    if q_case in c_case_counts:
      # If the key is already in the dictionary, increment its count
      c_case_counts[q_case] += 1
    else:
      # If the key is not in the dictionary, add it with a count of 1
      c_case_counts[q_case] = 1

In [ ]:
# Compare each digit in the answer. Returns a A45 pattern where 4 each digit means a failed digit
def get_digit_accuracy_impact(a_int, answer_str):
  a_str = str(a_int.cpu().numpy()).zfill(cfg.n_digits+1)
  match_str = "A"
  for i in range(cfg.n_digits+1):
    match_str += "" if answer_str[i] == a_str[i] else str(cfg.n_digits-i)

  return "" if match_str == "A" else match_str

In [ ]:
# Build up a list of success/failure digit-patterns found, and the frequency of each pattern
c_pattern_fails = {}


def clear_pattern_fails():
  global c_pattern_fails

  c_pattern_fails = {}


def add_pattern_fail(match_str):
  global c_pattern_fails

  if match_str in c_pattern_fails:
    # If the key is already in the dictionary, increment its count
    c_pattern_fails[match_str] += 1
  else:
    # If the key is not in the dictionary, add it with a count of 1
    c_pattern_fails[match_str] = 1


def get_pattern_fails():
  global c_pattern_fails

  results = ""
  top_result = ""
  if len(c_pattern_fails) > 0 :
    sorted_fails = dict(sorted(c_pattern_fails.items(), key=lambda item: item[1], reverse=True))
    for key, value in sorted_fails.items():
      this_cell = key + "=" + str(value)

      results = results + this_cell + " "

      if top_result == "":
        top_result = this_cell
      else:
        top_result = top_result + ", " + this_cell

  return results, top_result


def get_pattern_fails_total():
  global c_pattern_fails

  if len(c_pattern_fails) == 0:
    return 0

  total_sum = 0
  for key, value in c_pattern_fails.items():
      if isinstance(value, int):
          total_sum += value
  return total_sum

In [ ]:
# Build up a count of failure cases
c_case_fails = {}


def clear_case_fails():
  global c_case_fails

  c_case_fails = {}


def add_case_fail(case_key):
  global c_case_fails

  if case_key in c_case_fails:
    # If the key is already in the dictionary, increment its count
    c_case_fails[case_key] += 1
  else:
    # If the key is not in the dictionary, add it with a count of 1
    c_case_fails[case_key] = 1


def total_case_fails():
  global c_case_fails

  answer = 0
  for _, value in c_case_fails.items():
    answer = answer + value
  return answer


def get_case_fails():
  global c_case_fails
  global c_case_counts

  results = ""
  num_results = len(c_case_fails)
  if num_results > 0:
    sorted_fails = dict(sorted(c_case_fails.items(), key=lambda item: item[1], reverse=True))

    for key, value in sorted_fails.items():
      percent = round(100 * value / c_case_counts[key])
      results = results + "%" + key + "=" + str(percent)+ " "

  return results

In [ ]:
def predict_experiment_question(questions, the_hook, the_threshold):

  c_loss_mean = 0

  clear_case_fails()
  clear_pattern_fails()
  count_question_cases(questions)

  for question_num in range(questions.shape[0]):
    q = questions[question_num]

    model.reset_hooks()
    model.set_use_attn_result(True)
    exp_logits = model.run_with_hooks(q.cuda(), return_type="logits", fwd_hooks=the_hook)

    q_2d = q.unsqueeze(0)
    exp_losses_raw, exp_max_indices = logits_to_tokens_loss(exp_logits, q_2d.cuda())
    c_loss_mean = utils.to_numpy(loss_fn(exp_losses_raw).mean())

    # Only show the question if the loss exceeds the threshold (because of the ablated token position)
    if c_loss_mean > the_threshold:
      exp_answer_str = prediction_to_string(exp_max_indices)

      i = 12
      a = q[i+0] * 100000 + q[i+1] * 10000 + q[i+2] * 1000 + q[i+3] * 100 + q[i+4] * 10 + q[i+5] * 1;

      match_str = get_digit_accuracy_impact( a, exp_answer_str )
      # Only count the question if the model got the question wrong
      if 'A' in match_str:
        the_case = get_question_case(q)
        add_case_fail(the_case)
        add_pattern_fail(match_str)
        if verbose:
          print(tokens_to_string(q), "ModelAnswer:", exp_answer_str, "Matches:", match_str, "Loss:", c_loss_mean, "Case:", the_case )

  return c_loss_mean

# Part 12: Ablate ALL Heads in EACH token position. What is the impact on Loss?

Here we ablate all heads in each token position (overriding the model memory aka residual stream) and see if loss increases. If loss increases the token position is used by the algorithm. Unused token positions can be excluded from further analysis. Use "C_" prefix

In [ ]:
class C_Config():
  position : int = 0  # zero-based token position to ablate
  threshold : float = 0.01
  questions = varied_questions
  output = PrettyTable()
  perc_list = []
  hook_calls : int = 0

  min_useful_position : int = -1 # Minimum useful position where loss increases on ablation
  max_useful_position : int = -1 # Maximum useful position where loss increases on ablation


ccfg = C_Config()
ccfg.output.field_names = ["Position", "Fails", "% Fails by Case", "# Fails by Patterns"]

In [ ]:
verbose = False


def c_set_resid_post_hook(value, hook):
  global ccfg

  #print( "In hook", l_hook_resid_post_name[ccfg.layer], ccfg.ablate, ccfg.position, value.shape) # Get [64, 18, 510] = cfg.batch_size, num_tokens, d_model

  # Copy the mean resid post values in position N to all the batch questions
  value[:,ccfg.position,:] = mean_resid_post[0,ccfg.position,:].clone()


num_questions = 0
if cfg.n_digits >= 5 :
  c_fwd_hooks = [(l_hook_resid_post_name[0], c_set_resid_post_hook)] if cfg.n_layers == 1 else [(l_hook_resid_post_name[0], c_set_resid_post_hook),(l_hook_resid_post_name[1], c_set_resid_post_hook)]

  num_questions = ccfg.questions.shape[0]

  for ccfg.position in range(cfg.n_ctx):
    loss_mean = predict_experiment_question(ccfg.questions, c_fwd_hooks, ccfg.threshold)

    num_fails = total_case_fails()
    perc_fails = 0
    if num_fails > 0:
      perc_fails = round(100 * num_fails / num_questions)

      if ccfg.min_useful_position == -1:
        ccfg.min_useful_position = ccfg.position
      ccfg.max_useful_position = ccfg.position

    ccfg.perc_list = ccfg.perc_list + [perc_fails]

    (pattern_results, top_pattern) = get_pattern_fails()
    ccfg.output.add_row([str(ccfg.position), str(perc_fails)+"%", get_case_fails(), pattern_results])

In [ ]:
print_config()
print("num_questions=", num_questions, "min_useful_position=", ccfg.min_useful_position, "max_useful_position=", ccfg.max_useful_position )
print()

plt.hist(ccfg.perc_list, cfg.n_ctx, facecolor='blue', alpha=0.5)
plt.xlabel('Position')
plt.ylabel('Probability')
plt.title(r'Histogram of IQ: $\mu=100$, $\sigma=15$')
# Tweak spacing to prevent clipping of ylabel
plt.subplots_adjust(left=0.15)
plt.show()

print(ccfg.output.get_formatted_string(out_format=cfg.table_out_format))

# Part 13: Setup: Cell matrix

Uses "u_" prefix.

In [ ]:
class UsefulCell():
  # Is this cell an attention head? If not, it must be an MLP layer
  is_head: bool = True

  position: int = 0  # token-position
  layer: int = 0
  head: int = 0


# We (once) calculate the list of cells (attention head and MLP layers per position) that are useful to the model.
calc_useful_cells = True
# Once this list of useful cells is calculated (available) it is used to speed up functions.
useful_cells = []


def add_useful_cell(the_is_head, the_position, the_layer, the_head):
  global calc_useful_cells
  global useful_cells

  if calc_useful_cells:
    useful_cell = UsefulCell()
    useful_cell.is_head = the_is_head
    useful_cell.position = the_position
    useful_cell.layer = the_layer
    useful_cell.head = the_head

    useful_cells = useful_cells + [useful_cell]

In [ ]:
class U_Config():
  # This is a head+MLP (row) by token (column) matrix of percent of failure percentages with associated notes
  fail_percs = [[]]
  fail_notes = [[]]
  num_heads = 0
  num_mlps = 0


ucfg = U_Config()

In [ ]:
def ucfg_reset():
  global ucfg

  ucfg.fail_percs = [[0 for _ in range(cfg.n_ctx)] for _ in range((cfg.n_heads + 1) * cfg.n_layers)]
  ucfg.fail_notes = [["" for _ in range(cfg.n_ctx)] for _ in range((cfg.n_heads + 1) * cfg.n_layers)]
  ucfg.num_heads = 0
  ucfg.num_mlps = 0


ucfg_reset()

In [ ]:
def add_u_fail_perc( the_position, the_layer, the_head, perc_fails, notes ):
  global ucfg

  the_row = the_layer * (cfg.n_heads+1) + the_head

  if ucfg.fail_percs[the_row][the_position] == 0 :
    ucfg.fail_percs[the_row][the_position] = perc_fails

    add_useful_cell(the_head != cfg.n_heads, the_position, the_layer, the_head)

  else:
    print( "add_u_fail_perc: Bad index", the_row, the_position)

  ucfg.fail_percs[the_row][the_position] = perc_fails
  ucfg.fail_notes[the_row][the_position] = notes


def add_head_fail_perc( the_position, the_layer, the_head, perc_fails, notes ):
  global ucfg

  add_u_fail_perc( the_position, the_layer, the_head, perc_fails, notes )
  ucfg.num_heads += 1


def add_mlp_fail_perc( the_position, the_layer, perc_fails, notes ):
  global ucfg

  add_u_fail_perc( the_position, the_layer, cfg.n_heads, perc_fails, notes )
  ucfg.num_mlps += 1


def get_column_headings():
  datums = ["Position"]
  for i in range(ccfg.min_useful_position, ccfg.max_useful_position+1):
    datums = datums + ["P"+str(i)]
  return datums


def get_row_heading(i):
  head = i % (cfg.n_heads + 1)
  layer = i // (cfg.n_heads + 1)
  return ( "L" + str(layer) + "H" + str(head) ) if head < cfg.n_heads else "MLP "


# Print a 2 by 2 matrix of the percentage failures.
def print_u_fail_percs():
  global ucfg
  global mcfg

  print("The % failure rate when each head or MLP in each position is ablated, # failed heads =", ucfg.num_heads, ", # failed mlps =", ucfg.num_mlps )

  cell_output = PrettyTable()

  col_headings = get_column_headings()
  cell_output.field_names = col_headings

  num_rows = (cfg.n_heads + 1) * cfg.n_layers
  num_cols = ccfg.max_useful_position - ccfg.min_useful_position + 1
  percs_matrix = torch.zeros((num_rows, num_cols)).to(torch.int64)
  mask_matrix = torch.zeros((num_rows, num_cols)).to(torch.int64)
  row_headings = []

  for i in range(num_rows):
    row_heading = get_row_heading(i)
    row_headings = row_headings + [row_heading]

    datums = [row_heading]
    for j in range(num_cols):
      value = ucfg.fail_percs[i][ccfg.min_useful_position + j]
      datums = datums + ["" if value == 0 else str(value)+"%"]
      percs_matrix[i,j] = value
      mask_matrix[i,j] = ( value == 0 )

    cell_output.add_row(datums)

  # Display a 2D heat map of the percentages
  if use_sns == True:

    sns.set_theme(rc={"figure.dpi": 96}) # use higher resolution
    # %config InlineBackend.figure_format = "svg"
    sns.set(font_scale=0.8)
    sns.heatmap(utils.to_numpy(percs_matrix), annot=True, mask=utils.to_numpy(mask_matrix), cmap="YlGnBu", xticklabels=col_headings[1:], yticklabels=row_headings)
    plt.show()
  else:
    # Display a "pretty" table in html for use in blog
    print(cell_output.get_formatted_string(out_format=cfg.table_out_format))


# Print a 2 by 2 matrix of notes.
def print_u_fail_notes():
  global ucfg
  global mcfg

  print("The most common failure pattern (with associated failure #) when each head or MLP in each position is ablated")

  cell_output = PrettyTable()
  cell_output.field_names = get_column_headings()

  for i in range((cfg.n_heads + 1) * cfg.n_layers):
    datums = [get_row_heading(i)]
    for j in range(ccfg.min_useful_position, ccfg.max_useful_position+1):
      datums = datums + [ucfg.fail_notes[i][j]]
    cell_output.add_row(datums)

  print(cell_output.get_formatted_string(out_format=cfg.table_out_format))

# Part 14: Setup: Ablate each MLP in EACH position. Impact on Loss?
Ablating the MLP in each layer in each position and seeing if the loss increases shows which head+layer+MLP are used by the algorithm. Use "m_" prefix.

In [ ]:
class M_Config():
  position : int = 0  # zero-based token-position to ablate
  layer : int = 0 # zero-based layer to ablate. 0 to 1
  threshold : float = 0.12
  questions = varied_questions
  output = PrettyTable()
  hook_calls : int = 0


mcfg = M_Config()


def m_reset():
  global mcfg

  mcfg.output = PrettyTable()
  mcfg.output.field_names = ["Position", "MLP Layer", "% Fails", "% Fails by Case", "# Fails by Patterns"]
  mcfg.hook_calls = 0


def m_mlp_hook_post(value, hook):
  global mcfg

  mcfg.hook_calls += 1
  #print( "In m_mlp_hook_post", value.shape) # Get [1, 18, 2040] = ???, cfg.n_ctx, ???

  # Mean ablate. Copy the mean resid post values in position N to the MLP
  value[:,mcfg.position,:] =  mean_mlp_hook_post[:,mcfg.position,:].clone()


def m_perform_core(show_all = False):
  global mcfg

  the_hook = [(l_mlp_hook_post_name[mcfg.layer], m_mlp_hook_post)]
  loss_mean = predict_experiment_question(mcfg.questions, the_hook, mcfg.threshold)

  num_fails = total_case_fails()
  if show_all or (num_fails > 0):
    perc_fails = round(100 * num_fails / mcfg.questions.shape[0])
    (pattern_results, top_pattern) = get_pattern_fails()

    mcfg.output.add_row([str(mcfg.position), str(mcfg.layer), perc_fails, get_case_fails(), pattern_results])

    add_mlp_fail_perc( mcfg.position, mcfg.layer, perc_fails, top_pattern )


def m_perform(all_cells):
  global mcfg

  if cfg.n_digits >= 5 :
    ucfg_reset()
    m_reset()
    if all_cells:
      for mcfg.position in range(cfg.n_ctx):
        for mcfg.layer in range(cfg.n_layers):
          m_perform_core()
    else:
      for useful_cell in useful_cells:
        if not useful_cell.is_head:
          mcfg.position = useful_cell.position
          mcfg.layer = useful_cell.layer
          m_perform_core()


def m_print_results(title):
    global mcfg

    print_config()
    print()
    print(title, mcfg.questions.shape[0])
    print(mcfg.output.get_formatted_string(out_format=cfg.table_out_format))

# Part 15: Setup: Ablate EACH head in EACH position. Impact on Digit & Task Loss?
Ablating each head in each layer in each position and seeing if the loss increases shows which position+layer+head are used by the algorithm. Use "h_" prefix.

In [ ]:
class H_Config():
  position : int = 0 # zero-based token position to ablate. 0 to say 17
  layer : int = 0 # zero-based layer to ablate. 0 to 1
  head : int = 0 # zero-based head to ablate. 0 to 2
  threshold : float = 0.12
  questions = varied_questions
  output = PrettyTable()
  hook_calls: int = 0


hcfg = H_Config()


def h_reset():
  global hcfg

  hcfg.output = PrettyTable()
  hcfg.output.field_names = ["Position", "Layer", "Head", "% Fails", "% Fails by Case", "# Fails by Impact"]
  hcfg.hook_calls = 0


def h_set_attn_hook_z(value, hook):
  global hcfg

  hcfg.hook_calls += 1
  # print( "In h_set_attn_hook_z", value.shape) # Get [1, 18, 3, 170] = ???, cfg.n_ctx, cfg.n_heads, d_head

  # Mean ablate. Copy the mean resid post values in position N to all the batch questions
  value[:,hcfg.position,hcfg.head,:] = mean_attn_z[:,hcfg.position,hcfg.head,:].clone()


def h_perform_core(show_all = False):
  global hcfg

  the_hook = [(l_attn_hook_z_name[hcfg.layer], h_set_attn_hook_z)]
  loss_mean = predict_experiment_question(hcfg.questions, the_hook, hcfg.threshold)

  num_fails = total_case_fails()
  if show_all or (num_fails > 0):
    perc_fails = round(100 * num_fails / hcfg.questions.shape[0])
    (pattern_results, top_pattern) = get_pattern_fails()

    hcfg.output.add_row([str(hcfg.position), str(hcfg.layer), str(hcfg.head), perc_fails, get_case_fails(), pattern_results])

    add_head_fail_perc( hcfg.position, hcfg.layer, hcfg.head, perc_fails, top_pattern)

  return num_fails


def h_perform(all_cells):
  global hcfg

  if cfg.n_digits >= 5 :
    h_reset()
    if all_cells:
      for hcfg.position in range(ccfg.min_useful_position, ccfg.max_useful_position+1):
        for hcfg.layer in range(cfg.n_layers):
          for hcfg.head in range(cfg.n_heads):
            h_perform_core()
    else:
      for useful_cell in useful_cells:
        if useful_cell.is_head:
          hcfg.position = useful_cell.position
          hcfg.layer = useful_cell.layer
          hcfg.head = useful_cell.head
          h_perform_core()


def h_print_results(title, the_format=""):
  global hcfg

  print_config()
  print()
  print(title, hcfg.questions.shape[0], ", #hook_calls=", hcfg.hook_calls)

  if the_format == "":
    the_format = cfg.table_out_format
  print(hcfg.output.get_formatted_string(out_format=the_format))

# Part 16: Calculate show cell matrixes

Show the percentage failure rate (incorrect prediction) when individual Attention Heads and MLPs are ablated.

In [ ]:
def calc_cell_matrices(title, questions, all_cells):
  global mcfg
  global hcfg
  global verbose

  mcfg.questions = questions
  m_perform(all_cells)
  if verbose:
    m_print_results(title)

  hcfg.questions = questions
  h_perform(all_cells)
  if verbose:
    h_print_results(title)

In [ ]:
def print_cell_matrices():
  global verbose

  verbose = False

  print_config()
  print()
  print_u_fail_percs()
  print()
  print_u_fail_notes()

In [ ]:
def run_cell_matrices():
  global calc_useful_cells

  calc_useful_cells = True
  calc_cell_matrices("Varied questions", varied_questions, True)
  calc_useful_cells = False

  print_cell_matrices()


run_cell_matrices()

# Part 17 - Case Analysis
Just processing BA, MC, US9 questions

In [ ]:
calc_cell_matrices("BA questions", make_ba_questions(), False)

print_cell_matrices()

In [ ]:
calc_cell_matrices("UC1 questions", make_uc1_questions(), False)

print_cell_matrices()

In [ ]:
calc_cell_matrices("Simple US9 questions", make_simple_us9_questions(), False)

print_cell_matrices()

In [ ]:
calc_cell_matrices("Cascade US9 questions", make_cascade_us9_questions(), False)

print_cell_matrices()

#Part 18: SetUp: Calc and graph PCA decomposition

In [ ]:
tn_questions = 100

# These are n_digit addition questions where the first test_digits add up from 0 to 8
# Randomise the last test_digits-1 digits of both numbers
def make_t8_questions(test_digit):
    limit = 10 ** test_digit
    questions = []
    for i in range(tn_questions):
        x = random.randint(0, 8)
        y = random.randint(0, 8-x)
        x = x * limit + random.randint(0, limit-1)
        y = y * limit + random.randint(0, limit-1)
        questions.append([x, y])
    return make_questions(questions)


# These are n_digit addition questions where the first test_digits add up to 9
# Randomise the last test_digits-1 digits of both numbers
def make_t9_questions(test_digit):
    limit = 10 ** test_digit
    questions = []
    for i in range(tn_questions):
        x = random.randint(0, 9)
        y = 9 - x
        x = x * limit + random.randint(0, limit-1)
        y = y * limit + random.randint(0, limit-1)
        questions.append([x, y])
    return make_questions(questions)


# These are n_digit addition questions where the first test_digits add up to 10 to 18
# Randomise the last test_digits-1 digits of both numbers
def make_t10_questions(test_digit):
    limit = 10 ** test_digit
    questions = []
    for i in range(tn_questions):
        x = random.randint(1, 9)
        y = random.randint(10-x, 9)
        x = x * limit + random.randint(0, limit-1)
        y = y * limit + random.randint(0, limit-1)
        questions.append([x, y])
    return make_questions(questions)


def make_tricase_questions(test_digit):
  q1 = make_t8_questions(test_digit)
  q2 = make_t9_questions(test_digit)
  q3 = make_t10_questions(test_digit)

  questions = torch.vstack((q1, q2, q3))

  return questions

In [ ]:
# Do one Principal Component Analysis
def calc_tricase_pca(t_position, t_layer, t_head, t_digit):
  global tn_questions

  t_questions = make_tricase_questions(t_digit)
  #print('Sample t8 question:', t_questions[0].tolist())
  #print('Sample t9 question:', t_questions[tn_questions].tolist())
  #print('Sample t10 question:', t_questions[2*tn_questions].tolist())

  t_logits, t_cache = model.run_with_cache(t_questions)

  # Gather attention patterns for all the (randomly chosen) questions
  attention_outputs = []
  for i in range(len(t_questions)):

    # Output of individual heads, without final bias
    attention_cache=t_cache["result", t_layer, "attn"] # Output of individual heads, without final bias
    attention_output=attention_cache[i]  # Shape [n_ctx, n_head, d_model]
    attention_outputs.append(attention_output[t_position, t_head, :])

  attn_outputs = torch.stack(attention_outputs, dim=0).cpu()

  pca = PCA(n_components=6)
  pca.fit(attn_outputs)
  pca_attn_outputs = pca.transform(attn_outputs)

  title = 'P' + str(t_position) + '.L' + str(t_layer) + '.H'+str(t_head) + ', A'+str(t_digit)

  return (pca, pca_attn_outputs, title)


# Plot one PCA scatter graph
def graph_pca(pca, pca_attn_outputs, ax, title):
  global tn_questions

  ax.scatter(pca_attn_outputs[:tn_questions, 0], pca_attn_outputs[:tn_questions, 1], color='red', label='0-8') # t8 questions
  ax.scatter(pca_attn_outputs[tn_questions:2*tn_questions, 0], pca_attn_outputs[tn_questions:2*tn_questions, 1], color='green', label='9') # t9 questions
  ax.scatter(pca_attn_outputs[2*tn_questions:, 0], pca_attn_outputs[2*tn_questions:, 1], color='blue', label='10-18') # t10 questionsset

  if title != "" :
    ax.set_title(title)

In [ ]:
# Graph the PCA of Sn.Ln.Hn's attention pattern, using T8, T9, T10 questions that differ in the An digit
def add_one_pca_subplot(ax, t_position, t_layer, t_head, t_digit):
  pca, pca_attn_outputs, title = calc_tricase_pca(t_position, t_layer, t_head, t_digit)
  graph_pca( pca, pca_attn_outputs, ax, title)

In [ ]:
def save_plt_to_file( full_title ):
  if cfg.save_graph_to_file:
    filename = full_title.replace(" ", "_").replace(",", "").replace(":", "_")  + '.png'
    plt.savefig(filename)

#Part 19: PCA decomposition tri-state results

Plot attention heads in the positions 8 to 16 with a clear "tri-state" response to (exactly) one An.

In [ ]:
if cfg.n_digits == 5 and cfg.n_layers == 2 and use_pca :

  # graph all useful early cells
  fig, axs = plt.subplots(4, 2)
  fig.set_figheight(8)
  fig.set_figwidth(5)

  # Plot all useful attention heads in the positions 8 to 12 with the clearest An selected
  add_one_pca_subplot(axs[0, 0], 8, 0, 1, 2)    # S8.L0.H1 is clear only for A2
  add_one_pca_subplot(axs[0, 1], 9, 0, 1, 1)    # S9.L0.H1 is clear only for A1
  add_one_pca_subplot(axs[1, 0], 11, 0, 1, 3)   # S11.L0.H1 is clear only for A3
  add_one_pca_subplot(axs[1, 1], 11, 0, 2, 4)   # S11.L0.H2 is clear only for A4
  add_one_pca_subplot(axs[2, 0], 12, 0, 1, 3)   # S12.L0.H1 is clear only for A3
  add_one_pca_subplot(axs[2, 1], 13, 0, 1, 2)   # S13.L0.H1 is clear only for A2
  add_one_pca_subplot(axs[3, 0], 14, 0, 1, 1)   # S14.L0.H1 is clear only for A1

  lines_labels = [axs[0,0].get_legend_handles_labels()]
  lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
  fig.legend(lines, labels, loc='lower center', ncol=4)

  plt.tight_layout()
  save_plt_to_file('PCA_Trigrams')
  plt.show()

#Part 19B: PCA decomposition bi-state results

Plot attention heads in the positions 8 to 16 with a clear "bi-state" response to (exactly) one An.

In [ ]:
if cfg.n_digits == 5 and cfg.n_layers == 2 and use_pca :

  # graph all useful early cells
  fig, axs = plt.subplots(1, 2)
  fig.set_figheight(2)
  fig.set_figwidth(5)

  # Plot all useful attention heads in the positions 8 to 12 with the clearest An selected
  add_one_pca_subplot(axs[0], 10, 0, 1, 0)   # S10.L0.H1 is clear only for A0
  add_one_pca_subplot(axs[1], 15, 0, 1, 0)   # S15.L0.H1 is clear only for A0

  lines_labels = [axs[0].get_legend_handles_labels()]
  lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
  fig.legend(lines, labels, loc='lower center', ncol=4)

  plt.tight_layout()
  save_plt_to_file('PCA_Bigrams')
  plt.show()

In [ ]:
# Do one Principal Component Analysis and graph it
def run_one_tricase_pca(t_position, t_layer, t_head, t_digit):

  pca, pca_attn_outputs, title = calc_tricase_pca(t_position, t_layer, t_head, t_digit)

  # Plot the PCA results
  fig, ax = plt.subplots()
  graph_pca(pca, pca_attn_outputs, ax, "")

  full_title = 'PCA of attention: n_digits=' + str(cfg.n_digits) + ', ' + title
  plt.title(full_title + ', EVR[0]=' + str(round(pca.explained_variance_ratio_[0],3)) )

  plt.tight_layout()
  save_plt_to_file(full_title)
  plt.show()

  print( "First few principal components explain variance of:", pca.explained_variance_ratio_)

#Part 19C: PCA decomposition of useful cells with digits 0 to 4

Parts 19A and 19B are selective. This part is not. Use it to find (verify) the interesting parts.

In [ ]:
def graph_all_pca_results():

  for useful_cell in useful_cells:
      if useful_cell.is_head:
        position = useful_cell.position
        layer = useful_cell.layer
        head = useful_cell.head
        print( "PCA: position=", position, "layer=", layer, "head=", head)

        fig, axs = plt.subplots(3, 2)

        add_one_pca_subplot(axs[0, 0], position, layer, head, 0)
        add_one_pca_subplot(axs[0, 1], position, layer, head, 1)
        add_one_pca_subplot(axs[1, 0], position, layer, head, 2)
        add_one_pca_subplot(axs[1, 1], position, layer, head, 3)
        add_one_pca_subplot(axs[2, 0], position, layer, head, 4)

        plt.tight_layout()
        plt.show()


if use_pca :
  graph_all_pca_results()

# Part 20: Implement Mathematical framework

In [ ]:
def tri_case(dn1,dn2):
  s = dn1 + dn2
  if s >= 10:
    return 10
  if s == 9:
    return 9
  return 8


def tri_add(dn_cx, dm_cy):
  if dn_cx == 10 or (dn_cx == 9 and dm_cy == 10):
    return 10
  if dn_cx == 8 and dm_cy == 10:
    return 9
  return 8


def addition_psuedo_code( d4, d3, d2, d1, d0, d4d, d3d, d2d, d1d, d0d ):
  # D2.C1 | TriCase(D2, D2’) | P8.L0.H1 and P8.L0.MLP
  d2_c1 = tri_case(d2, d2d)

  # D1.C1 | TriCase(D1, D1') | P9.L0.H1 and P9.L0.MLP
  d1_c1 = tri_case(d1, d1d)

  # D1.C2 | TriAdd(D1.C1, TriCase(D0, D0’)) | P10.L0.H1 and P10.L0.MLP
  d1_c2 =  tri_add( d1_c1, tri_case(d0, d0d) )

  # D3.C4 | TriAdd(TriCase(D3, D3’), TriAdd(D2.C1,D1.C2)) | P11.L0.H1
  d3_c4 = tri_add( tri_case(d3, d3d), tri_add(d2_c1,d1_c2) )

  # D4.C1 | TriCase(D4, D4’) | P11.L0.H2
  d4_c1 = tri_case(d4, d4d)

  # D4.C5 | TriAdd(D4.C1, D3.C4) | P11.L0.MLP
  d4_c5 = tri_add(d4_c1, d3_c4)

  # A5 | (D4.C5 == 10) | P11.L1.MLP
  a5 = 1 if d4_c5 == 10 else 0

  # D4.BA | (D4 + D4') % 10 | P12.L0.H0 + H2
  d4_ba = (d4 + d4d) % 10

  # D3.C4 | TriAdd(TriCase(D3, D3’), TriAdd(D2.C1,D1.C2)) | P12.L0.H1
  d3_c4 = tri_add( tri_case(d3, d3d), tri_add(d2_c1, d1_c2) )

  # A4 | (D4.BA + D3.C4 % 10) % 10 | P12.L0.MLP and P12.L1.MLP
  a4 = (d4_ba + (d3_c4 % 10)) % 10

  #D3.BA | (D3 + D3') % 10 | P13.L0.H0 + H2
  d3_ba = (d3 + d3d) % 10

  # D2.C3 | TriAdd(D2.C1,D1.C2) | P13.L0.H1
  d2_c3 = tri_add(d2_c1, d1_c2)

  # A3 | (D3.BA + D2.C3 % 10) % 10 | P13.L0.MLP and P13.L1.MLP
  a3 = (d3_ba + (d2_c3 % 10)) % 10

  # D2.BA | (D2 + D2') % 10 | P14.L0.H0 + H2
  d2_ba = (d2 + d2d) % 10

  # D1.C2 | Copy from P10 | P14.L0.H1
  # skip

  # A2 | (D2.BA + D1.C2 % 10) % 10 | P14.L0.MLP and P14.L1.MLP
  a2 = (d2_ba + (d1_c2 % 10)) % 10

  # D1.BA | (D1 + D1') % 10 | P15.L0.H0 + H2
  d1_ba = (d1 + d1d) % 10

  # D0.MC | (D0 + D0') // 10 | P15.L0.H1
  d0_mc = (d0 + d0d) // 10

  # A1 | (D1.BA + D0.MC) % 10 | P15.L0.MLP and P15.L1.MLP
  a1 = (d1_ba + d0_mc) % 10

  # A0 | (D0 + D0') % 10 | P16.L0.H0 + H2 P16.L0.MLP and P16.L1.MLP
  a0 = (d0 + d0d) % 10

  return a5, a4, a3, a2, a1, a0


def do_addition_question(question):
  if cfg.n_digits == 5:
    d4 = int(question[0])
    d3 = int(question[1])
    d2 = int(question[2])
    d1 = int(question[3])
    d0 = int(question[4])
    d4d = int(question[6])
    d3d = int(question[7])
    d2d = int(question[8])
    d1d = int(question[9])
    d0d = int(question[10])

    a5, a4, a3, a2, a1, a0 = addition_psuedo_code( d4, d3, d2, d1, d0, d4d, d3d, d2d, d1d, d0d)

    d = d4 * 10000 + d3 * 1000 + d2 * 100 + d1 * 10 + d0
    dd = d4d * 10000 + d3d * 1000 + d2d * 100 + d1d * 10 + d0d
    a = a5 * 100000 + a4 * 10000 + a3 * 1000 + a2 * 100 + a1 * 10 + a0

    if d + dd != a :
      print(d4, d3, d2, d1, d0, d4d, d3d, d2d, d1d, d0d)
      print("Bad addition:", d, "+", dd, "=", a, "Should be", d+dd, "Delta", d+dd-a)
      return 1

    return 0

In [ ]:
if cfg.n_digits == 5:
  num_questions = varied_questions.shape[0]
  print("#Questions=", num_questions)

  num_fails = 0
  for i in range(3): #num_questions):
    question = varied_questions[i]
    num_fails += do_addition_question(question)

  print("#Fails=", num_fails)

#Part 21: MLP Visualisation

In [ ]:
import einops
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#from IPython.display import clear_output
#import circuitsvis as cv
#import h5py
#import palettable

cache = sample_cache

attn_mat = cache['pattern', 0, 'attn'][:,:,-3,:]
neuron_acts_pre = cache['blocks.0.mlp.hook_pre'][:,-3]
neuron_acts_post = cache['blocks.0.mlp.hook_post'][:,-3]
print(neuron_acts_post.shape)
neuron_acts_post_sq = einops.rearrange(neuron_acts_post, "(x y) d_mlp -> x y d_mlp", x=239) # num questions
neuron_acts_pre_sq = einops.rearrange(neuron_acts_pre, "(x y) d_mlp -> x y d_mlp", x=239) # num questions
#posmask = data.tok[:,-2] == 100
#posmask = data.tok[:,-2] == 100
#negmask = data.tok[:,-2] == 101

#posmask_sq = einops.rearrange(posmask.numpy(), "(x y) -> x y ", x=239)[:,:,None,None]
#negmask_sq = einops.rearrange(negmask.numpy(), "(x y) -> x y ", x=239)[:,:,None,None]
#print(posmask_sq.shape, posmask.sum(), negmask.sum())

def plot_neuronact(pos: int):
    #pq clear_output()
    #pq data = neuron_acts_pre_sq.detach().numpy()[:,:,pos]
    data = neuron_acts_pre_sq.cpu().numpy()[:,:,pos]
    ft2d = np.fft.fft2(data)
    power = (ft2d*np.conj(ft2d)).real

    fig, axs = plt.subplots(1, 2, figsize=(12,6))
    plot = axs[0].imshow(data, cmap='magma', vmin=0)#, vmax=1)
    cbar = plt.colorbar(plot, fraction=0.1)
    cbar.set_label(r'neuron activation strength {}'.format(pos))
    axs[0].set_ylim(-0.5, 99.5)
    axs[0].set_yticks(range(100), labels=range(100), size=5.5);
    axs[0].set_xticks(range(100), labels=range(100), size=5.5, rotation='vertical');

    plot = axs[1].imshow(np.log10(power), cmap='magma', vmin=0)#, vmax=1)
    cbar = plt.colorbar(plot, fraction=0.1)
    cbar.set_label(r'neuron log10 power {}'.format(pos))
    #axs[1].set_ylim(-0.5, 99.5)
    #axs[1].set_yticks(range(100));
    #axs[1].set_yticklabels([f'{freq:.2f}' for freq in freqs], fontsize=5.5);
    #axs[1].set_xticks(range(100));
    #axs[1].set_xticklabels([f'{freq:.2f}' for freq in freqs], fontsize=5.5, rotation='vertical');

interact(plot_neuronact, pos=widgets.IntText(value=0, description='Index:'))

# Part 22 : What does the model do at S11 (the "=" token)? NOT USED

NOT USED

At S11, the model predicts A5 (P12), using attention heads paying attention to tokens at P0.. P4, P6 .. P10. TBC  

Use "a_" prefix

In [ ]:
class A_Config():
  prediction_step : int = 11 # The prediction step we want to focus on. P11 is the "=" token

  token_position : int = 8 # The token position we want to get/set. P8 to P11 contribute to A5 calculations
  layer : int = 0 # The layer we want to get/set
  head : int = 1 # The head we want to get/set
  answer_digit : int = '5'  # The answer digit we are testing
  threshold : int = 0.01

  hook_calls: int = 0
  answer_failures : int = 0    # Failures of any digit
  digit_failures : int = 0     # Failures of the answer_digit

  questions = []
  store = []


acfg = A_Config()

In [ ]:
# Get and put attention head values

def a_null_attn_z_hook(value, hook):
  global acfg

  acfg.hook_calls += 1
  #print("In a_null_attn_z_hook", value.shape)  # Get [1, 18, 3, 170] = ???, cfg.n_ctx, cfg.n_heads, cfg.d_head


def a_get_l0_attn_z_hook(value, hook):
  global acfg

  if acfg.layer == 0:
    acfg.hook_calls += 1
    # print( "In a_get_l0_attn_z_hook", value.shape) # Get [1, 18, 3, 170] = ???, cfg.n_ctx, cfg.n_heads, cfg.d_head
    acfg.store = value.clone()


def a_get_l1_attn_z_hook(value, hook):
  global acfg

  if acfg.layer == 1:
    acfg.hook_calls += 1
    # print( "In acfg.get_l1_attn_z_hook", value.shape) # Get [1, 18, 3, 170] = ???, cfg.n_ctx, cfg.n_heads, cfg.d_head
    acfg.store = value.clone()


def a_put_l0_attn_z_hook(value, hook):
  global acfg

  if acfg.layer == 0:
    acfg.hook_calls += 1
    # print( "In a_l0_attn_z_hook", value.shape) # Get [1, 18, 3, 170] = ???, cfg.n_ctx, cfg.n_heads, d_head
    value[:,acfg.token_position,acfg.head,:] = acfg.store[:,acfg.token_position,acfg.head,:].clone()



def a_put_l1_attn_z_hook(value, hook):
  global acfg

  if acfg.layer == 1:
    acfg.hook_calls += 1
    # print( "In a_put_l1_attn_z_hook", value.shape) # Get [1, 18, 3, 170] = ???, cfg.n_ctx, cfg.n_heads, d_head
    value[:,acfg.token_position,acfg.head,:] = acfg.store[:,acfg.token_position,acfg.head,:].clone()

In [ ]:
def a_predict_questions_with_hooks(the_hooks):
  global acfg
  global model

  acfg.hook_calls = 0
  acfg.answer_failures = 0
  acfg.digit_failures = 0
  sum_loss = 0.0

  for question_num in range(acfg.questions.shape[0]):
    q = acfg.questions[question_num]

    model.reset_hooks()
    model.set_use_attn_result(True)
    exp_logits = model.run_with_hooks(q.cuda(), return_type="logits", fwd_hooks=the_hooks)

    q_2d = q.unsqueeze(0)
    exp_losses_raw, exp_max_indices = logits_to_tokens_loss(exp_logits, q_2d.cuda())
    loss_mean = utils.to_numpy(loss_fn(exp_losses_raw).mean())
    sum_loss += loss_mean

    # Only show the question if the loss exceeds the threshold (because of the ablated step)
    if loss_mean > acfg.threshold:
      acfg.answer_failures += 1

      exp_answer_str = prediction_to_string(exp_max_indices)

      i = 12
      a = q[i+0] * 100000 + q[i+1] * 10000 + q[i+2] * 1000 + q[i+3] * 100 + q[i+4] * 10 + q[i+5] * 1;

      match_str = get_digit_accuracy_impact( a, exp_answer_str )
      acfg.digit_failures += 1 if acfg.answer_digit in match_str else 0
      if verbose:
        print("    "+match_str)


  return sum_loss

In [ ]:
def explain_A5():
  global acfg

  if cfg.n_digits == 5 and cfg.n_layers == 2:

    acfg.questions = varied_questions[:7]
    acfg.answer_digit = '5'
    print("Explain_A5: #questions=", acfg.questions.shape[0], "answer_digit=", acfg.answer_digit)

    # With no hook, loss should be zero
    # How many times is the hook called? cfg.n_ctx? If so this sequence gives a_prediction_step
    the_null_attn_z_hook = [(l_attn_hook_z_name[0], a_null_attn_z_hook)]
    sum_loss = a_predict_questions_with_hooks(the_null_attn_z_hook)
    print("Run with null hook: #Calls=", acfg.hook_calls, "#AnswerFails=", acfg.answer_failures, "#DigitFails=", acfg.digit_failures, "sum_loss=", sum_loss)


    the_get_attn_z_hook = [(l_attn_hook_z_name[0], a_get_l0_attn_z_hook),(l_attn_hook_z_name[1], a_get_l1_attn_z_hook)]
    sum_loss = a_predict_questions_with_hooks(the_get_attn_z_hook)
    print("Run with get hook: #Calls=", acfg.hook_calls, "#AnswerFails=", acfg.answer_failures, "#DigitFails=", acfg.digit_failures, "store.shape=", acfg.store.shape, "sum_loss=", sum_loss)


    acfg.prediction_step = 11 # Not used
    acfg.token_position = 10
    acfg.layer = 0
    acfg.head = 1
    acfg.store = mean_attn_z.clone() # Mean ablate

    the_put_attn_z_hook = [(l_attn_hook_z_name[0], a_put_l0_attn_z_hook),(l_attn_hook_z_name[1], a_put_l1_attn_z_hook)]
    #sum_loss = a_predict_questions_with_hooks(the_put_attn_z_hook)
    #print("Run with put hook: #Calls=", acfg.hook_calls, "#AnswerFails=", acfg.answer_failures, "#DigitFails=", acfg.digit_failures, "sum_loss=", sum_loss)


    # Repeatedly predict these questions, mean ablating one token_position+head+layer at a time, to see the impact on A5
    print()
    for acfg.token_position in range(8,12):
      for acfg.layer in range(cfg.n_layers):
          for acfg.head in range(cfg.n_heads):
            sum_loss = a_predict_questions_with_hooks(the_put_attn_z_hook)
            if acfg.answer_failures > 0:
              temp = "#Digit" + str(acfg.answer_digit) + "Fails="
              print("Run with put hook: token_position=", acfg.token_position, "layer=", acfg.layer, "head=", acfg.head, "#AnswerFails=", acfg.answer_failures, temp, acfg.digit_failures, "sum_loss=", sum_loss)
            #else:
            #  print("Good: token_position=", acfg.token_position, "layer=", acfg.layer, "head=", acfg.head, "sum_loss=", sum_loss)


#explain_A5()